<a href="https://colab.research.google.com/github/harenlin/PySpark-Learning/blob/main/SQLOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [9]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/PySpark/Datasets/'
crime = spark.read.csv(path + 'rec-crime-pfa.csv', inferSchema=True, header=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
crime.show(5)

+----------------+-----------------+----------+--------------------+-------------------------------------+
|12 months ending|              PFA|    Region|             Offence|Rolling year total number of offences|
+----------------+-----------------+----------+--------------------+-------------------------------------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|                                25959|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft|                                 3090|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|                                26202|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|                                    2|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|                                14561|
+----------------+-----------------+----------+--------------------+-------------------------------------+
only showing top 5 rows



In [11]:
print(crime.printSchema())

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling year total number of offences: integer (nullable = true)

None


In [12]:
# So, in order for us to perform SQL calls off of this dataframe, 
# we will need to rename any variables that have spaces in them. 
# We will not be using the first variable so I'll leave that one as is, 
# but we will be using the last variable, so I will go ahead and change that to Count so we can work with it. 
df = crime.withColumnRenamed('Rolling year total number of offences','Count') #.withColumn("12 months ending", crime["12 months ending"].cast(DateType())).
print(df.printSchema())

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Count: integer (nullable = true)

None


# Spark SQL

In [13]:
# Create a temporary view of the dataframe
df.createOrReplaceTempView("tempview")

In [14]:
# Then Query the temp view
spark.sql("SELECT * FROM tempview WHERE Count > 1000").limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Count
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561
4,31/03/2003,Avon and Somerset,South West,Drug offences,2308


In [15]:
spark.sql("SELECT * \
                FROM tempview \
                WHERE Count > 1000 ").show(5)

+----------------+-----------------+----------+--------------------+-----+
|12 months ending|              PFA|    Region|             Offence|Count|
+----------------+-----------------+----------+--------------------+-----+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|
|      31/03/2003|Avon and Somerset|South West|       Drug offences| 2308|
+----------------+-----------------+----------+--------------------+-----+
only showing top 5 rows



In [16]:
spark.sql("SELECT Region, Offence \
                FROM tempview \
                WHERE Count > 20000 ").show(5)

+--------------------+--------------------+
|              Region|             Offence|
+--------------------+--------------------+
|          South West|All other theft o...|
|          South West|Criminal damage a...|
|          South West|    Vehicle offences|
|British Transport...|All other theft o...|
|          South West|All other theft o...|
+--------------------+--------------------+
only showing top 5 rows



In [17]:
# We can even do aggregated "group by" calls like this
spark.sql("SELECT Region, sum(Count) AS Total \
                FROM tempview \
                GROUP BY Region").limit(5).toPandas()

,Region,Total
0,Fraud: CIFAS,7678981
1,North West,30235732
2,British Transport Police,3029117
3,Wales,11137260
4,London,42691902


# SQL Transformer

In [19]:
from pyspark.ml.feature import SQLTransformer

In [20]:
# create an SQL call 
sqlTrans = SQLTransformer(
    statement = "SELECT PFA, Region, Offence FROM __THIS__") 
# Note that "__THIS__" is a special word and cannot be change to __THAT__ for example
# Also Note that a call like this won't work...
# SQLTransformer(statement = "SELECT PFA, Region, Offence FROM __THIS__").show()

# use it to transform our df object
sqlTrans.transform(df).show(5)

+-----------------+----------+--------------------+
|              PFA|    Region|             Offence|
+-----------------+----------+--------------------+
|Avon and Somerset|South West|All other theft o...|
|Avon and Somerset|South West|       Bicycle theft|
|Avon and Somerset|South West|Criminal damage a...|
|Avon and Somerset|South West|Death or serious ...|
|Avon and Somerset|South West|   Domestic burglary|
+-----------------+----------+--------------------+
only showing top 5 rows



In [21]:
type(sqlTrans)

pyspark.ml.feature.SQLTransformer

In [22]:
# Note that this call will not work on the original dataframe "crime" when the variable "Count" is a string
sqlTrans = SQLTransformer(statement = "SELECT Offence, SUM(Count) as Total FROM __THIS__ GROUP BY Offence") 
sqlTrans.transform(df).show(5)

+--------------------+--------+
|             Offence|   Total|
+--------------------+--------+
|Public order offe...|10925676|
|       Bicycle theft| 5297006|
|Residential burglary| 1671469|
|Violence without ...|16590158|
|All other theft o...|30979393|
+--------------------+--------+
only showing top 5 rows



In [23]:
sqlTrans = SQLTransformer(statement = "SELECT PFA, Offence FROM __THIS__ WHERE Count > 1000") 
sqlTrans.transform(df).show(5)

+-----------------+--------------------+
|              PFA|             Offence|
+-----------------+--------------------+
|Avon and Somerset|All other theft o...|
|Avon and Somerset|       Bicycle theft|
|Avon and Somerset|Criminal damage a...|
|Avon and Somerset|   Domestic burglary|
|Avon and Somerset|       Drug offences|
+-----------------+--------------------+
only showing top 5 rows



In [24]:
# first we need to read in the library
from pyspark.sql.functions import expr
# Method 1 
df.withColumn("percent", expr("round((count/244720928)*100,2)")).show(5)

+----------------+-----------------+----------+--------------------+-----+-------+
|12 months ending|              PFA|    Region|             Offence|Count|percent|
+----------------+-----------------+----------+--------------------+-----+-------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|   0.01|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090|    0.0|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|   0.01|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|    0.0|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|   0.01|
+----------------+-----------------+----------+--------------------+-----+-------+
only showing top 5 rows



In [25]:
# Method 2
df.select("*", expr("round((count/244720928)*100,2) AS percent")).show(5)

+----------------+-----------------+----------+--------------------+-----+-------+
|12 months ending|              PFA|    Region|             Offence|Count|percent|
+----------------+-----------------+----------+--------------------+-----+-------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|   0.01|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090|    0.0|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|   0.01|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|    0.0|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|   0.01|
+----------------+-----------------+----------+--------------------+-----+-------+
only showing top 5 rows



In [26]:
# Method 3
df.selectExpr("*", "round((count/244720928)*100,2) AS percent").show(5)

+----------------+-----------------+----------+--------------------+-----+-------+
|12 months ending|              PFA|    Region|             Offence|Count|percent|
+----------------+-----------------+----------+--------------------+-----+-------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|   0.01|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090|    0.0|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|   0.01|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|    0.0|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|   0.01|
+----------------+-----------------+----------+--------------------+-----+-------+
only showing top 5 rows

